In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [14]:
# This is basically the numerical representation of the audio file
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data,2,0)

This data is at (40, 862, 920) for current shape which is
40 coeffs at each of the 862 timesteps for 920 files
After moveaxis
Now the data is at 920 files with 40 coeffs for 862 time steps

In [15]:
df = pd.read_csv('primary_dataframe.csv')
y = df.diagnosis_Healthy
del df # Get this out of memory

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data,y,test_size = 0.2, shuffle = True
)

X_train, X_validate, y_train, y_validate = train_test_split(
    X_train,y_train,test_size = 0.25, shuffle = True
)

In [17]:
print("Shapes of the data: ",X_train.shape, X_validate.shape, X_test.shape)
print("% Healthy: ",y_train.mean(), y_validate.mean(), y_test.mean())

Shapes of the data:  (552, 40, 862) (184, 40, 862) (184, 40, 862)
% Healthy:  0.041666666666666664 0.043478260869565216 0.021739130434782608


In [18]:
#Resample to help fix imbalance
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler


oversampler = ADASYN(sampling_strategy='auto')

X_flat = X_train.reshape(X_train.shape[0], -1)

X_oversampled, y_oversampled = oversampler.fit_resample(X_flat, y_train)

X_oversampled_3D = X_oversampled.reshape(-1, X_train.shape[1], X_train.shape[2])

X_trainr = X_oversampled_3D
y_trainr = y_oversampled

The code above should bring the % Healthy up to 50%

In [23]:
m = data.max()
model = Sequential([
  layers.Rescaling(1./m, input_shape=(data.shape[1], data.shape[2],1)),
  layers.Conv2D(16, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Conv2D(32, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Conv2D(64, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Conv2D(64, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Conv2D(64, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1,activation = 'sigmoid')
])

In [24]:
######
def weighted_binary_cross_entropy(y_true, y_pred):
    # Define the weight for the minority class
    w = tf.constant(32.33, dtype=tf.float32)
    
    # Compute the binary cross-entropy loss for each example
    bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
    
    # Apply the weights to the loss
    weighted_bce = tf.keras.backend.mean((1 - w) * y_true * bce + w * (1 - y_true) * bce)
    
    return weighted_bce

In [26]:
model.compile(optimizer='adam',
              loss=weighted_binary_cross_entropy,
              metrics=['BinaryAccuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

1 = Healthy = positive
0 = Sick = negative

false negative = sick when healthy => need to track it

In [27]:
num_batch_size = 256
num_epochs = 250

model.fit(x=X_trainr, y=y_trainr, batch_size = num_batch_size,
          validation_data=(X_validate, y_validate), epochs=num_epochs)

Epoch 1/250
5/5 [==============================] - 7s 970ms/step - loss: -10.5977 - binary_accuracy: 0.5165 - precision_3: 0.5388 - recall_3: 0.2358 - val_loss: 3.2223 - val_binary_accuracy: 0.9565 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 2/250
5/5 [==============================] - 4s 811ms/step - loss: -148.6376 - binary_accuracy: 0.4995 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: -21.7946 - val_binary_accuracy: 0.9565 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 3/250
5/5 [==============================] - 4s 721ms/step - loss: -922.0554 - binary_accuracy: 0.4995 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: -130.2807 - val_binary_accuracy: 0.9565 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 4/250
5/5 [==============================] - 4s 759ms/step - loss: -4463.4062 - binary_accuracy: 0.4995 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: -622.9602 - val_binary_accuracy: 0.9565 -

KeyboardInterrupt: 

In [17]:
model.save('health_model.h5')

In [11]:
model = tf.keras.models.load_model('health_model.h5')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

